In [9]:
#!pip install splink

In [3]:
import duckdb
from tqdm.auto import tqdm 
import pandas as pd

connection_source = duckdb.connect(database="../dbt/database_name.duckdb", read_only = True)

In [4]:
# OK : Filtrer les propriétaires ayant un logements vacant dans la prod (lovac 2024)
# Logement est le lien entre un propriétaire national et départemental 
# Aller chercher les logements via les ID locaux dans les fichiers fonciers
# Aller chercher les logements via les ID locaux dans la prod 

In [ ]:
# Queries pour charger les DataFrames
query_owners = """ 
WITH ff_idlocal_idprocte AS (
    SELECT ff_idlocal, ff_idprocpte
    FROM raw_lovac_2024
    UNION ALL
    SELECT ff_idlocal, ff_idprocpte
    FROM raw_lovac_2023
),
ff_idprocte AS (
    SELECT
        ff_idprocpte,
        ff_idlocal
    FROM ff_idlocal_idprocte
),
ff_unique_owner_ids AS (
    SELECT
        o.ff_owner_idpersonne,
        unnest(o.ff_owner_idprocpte) AS idprocpte
    FROM main_int.int_ff_owners_dedup o
),
ff_data AS (
    SELECT oi.ff_owner_idpersonne, oi.idprocpte, ids.ff_idlocal
    FROM ff_unique_owner_ids oi
    LEFT JOIN ff_idprocte ids ON ids.ff_idprocpte = oi.idprocpte
),
prod_owners AS (
    SELECT
        o.id,
        ph.local_id
    FROM main_stg.stg_production_owners o
    JOIN production.ban_addresses ba
        ON ba.address_kind = 'Owner' AND ba.ref_id = o.id
    JOIN main_stg.stg_production_owners_housing poh
        ON poh.owner_id = o.id
    JOIN main_stg.stg_production_housing ph
        ON poh.housing_id = ph.id
    WHERE
        (occupancy_source = 'V' OR occupancy = 'V')
        AND data_file_years IS NOT NULL
        AND list_contains(data_file_years, 'lovac-2024')
        AND o.birth_date IS NOT NULL
        AND idpersonne IS NULL
),
data_to_check AS (
    SELECT prod_owners.id,
            fd.ff_owner_idpersonne,
            array_agg(fd.ff_idlocal) AS local_ids
    FROM prod_owners
    JOIN ff_data fd ON fd.ff_idlocal = prod_owners.local_id
    GROUP BY prod_owners.id, fd.ff_owner_idpersonne
), input_data AS (
SELECT dtc.id, dtc.ff_owner_idpersonne, dtc.local_ids
    , ipo.full_name, ipo.birth_date, ipo.postal_code, ipo.address_dgfip[0]
    , ifo.ff_owner_fullname, ifo.ff_owner_birth_date, ifo.ff_owner_firstname, ifo.ff_owner_lastname, ifo.ff_owner_postal_code
    , ifo.ff_owner_address_1 || ' ' || ifo.ff_owner_address_2 || ' ' || ifo.ff_owner_address_3 || ' ' || ifo.ff_owner_address_4 AS owner_address,

FROM data_to_check dtc
JOIN main_int.int_production_owners ipo ON ipo.id = dtc.id
JOIN main_int.int_ff_owners_dedup ifo ON ifo.ff_owner_idpersonne = dtc.ff_owner_idpersonne)
SELECT *
FROM input_data
"""


df_ff_owners = connection_source.execute(query_owners).fetchdf()
print(df_ff_owners.shape)

connection_source.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(2284662, 14)


In [6]:
import uuid

In [7]:
df_ff_owners["matching_id"] = df_ff_owners["id"].apply(lambda x: str(uuid.uuid4()))

In [8]:
df_ff_owners.columns

Index(['id', 'ff_owner_idpersonne', 'local_ids', 'full_name', 'birth_date',
       'postal_code', 'postal_code_1', 'ipo.address_dgfip[0]',
       'ff_owner_fullname', 'ff_owner_birth_date', 'ff_owner_firstname',
       'ff_owner_lastname', 'ff_owner_postal_code', 'owner_address',
       'matching_id'],
      dtype='object')

In [23]:
df_prod = df_ff_owners[["id", "matching_id", "full_name", "birth_date", "postal_code", "ipo.address_dgfip[0]"]].rename(
    columns = {
        "id": "unique_id",
        "full_name": "owner_fullname",
        "birth_date": "owner_birth_date",
        "postal_code": "owner_postal_code",
        "ipo.address_dgfip[0]": "owner_address"
    }
)
df_prod["owner_firstname"] = df_prod["owner_fullname"]
df_prod["owner_lastname"] = df_prod["owner_fullname"]

df_ff = df_ff_owners[["ff_owner_idpersonne","matching_id", "ff_owner_fullname", "ff_owner_lastname", "ff_owner_birth_date", "owner_address", "ff_owner_postal_code", "ff_owner_firstname"]].rename(
    columns={
        "ff_owner_idpersonne": "unique_id",
        "ff_owner_fullname": "owner_fullname",
        "ff_owner_lastname": "owner_lastname",
        "ff_owner_birth_date": "owner_birth_date",
        "owner_address": "owner_address",
        "ff_owner_postal_code": "owner_postal_code",
        "ff_owner_firstname": "owner_firstname"
    }
)

In [31]:
df_ff.shape

(2284662, 8)

In [30]:
df_ff.unique_id.unique().shape

(1056790,)

In [36]:
import splink.comparison_library as cl

first_name_comparison = cl.ExactMatch("first_name")

In [40]:
first_name_comparison.get_comparison("duckdb").as_dict()

{'output_column_name': 'first_name',
 'comparison_levels': [{'sql_condition': '"first_name_l" IS NULL OR "first_name_r" IS NULL',
   'label_for_charts': 'first_name is NULL',
   'is_null_level': True},
  {'sql_condition': '"first_name_l" = "first_name_r"',
   'label_for_charts': 'Exact match on first_name'},
  {'sql_condition': 'ELSE', 'label_for_charts': 'All other comparisons'}],
 'comparison_description': 'ExactMatch'}

In [43]:
settings.get_settings("duckdb").as_dict()

{'link_type': 'link_only',
 'probability_two_random_records_match': 0.0001,
 'retain_matching_columns': True,
 'retain_intermediate_calculation_columns': False,
 'additional_columns_to_retain': [],
 'sql_dialect': 'duckdb',
 'linker_uid': 'fy6xvrlq',
 'em_convergence': 0.0001,
 'max_iterations': 25,
 'bayes_factor_column_prefix': 'bf_',
 'term_frequency_adjustment_column_prefix': 'tf_',
 'comparison_vector_value_column_prefix': 'gamma_',
 'unique_id_column_name': 'unique_id',
 'source_dataset_column_name': 'source_dataset',
 'blocking_rules_to_generate_predictions': [{'blocking_rule': 'l."matching_id" = r."matching_id"',
   'sql_dialect': 'duckdb'}],
 'comparisons': [{'output_column_name': 'owner_fullname',
   'comparison_levels': [{'sql_condition': '"owner_fullname_l" IS NULL OR "owner_fullname_r" IS NULL',
     'label_for_charts': 'owner_fullname is NULL',
     'is_null_level': True},
    {'sql_condition': '"owner_fullname_l" = "owner_fullname_r"',
     'label_for_charts': 'Exact mat

In [24]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on

settings = SettingsCreator(
    link_type="link_only",
    blocking_rules_to_generate_predictions=[
        block_on("matching_id"),
    ],
    comparisons=[
        cl.NameComparison(
            "owner_fullname",
        ),
        cl.DateOfBirthComparison(
            "owner_birth_date",
            invalid_dates_as_null=True,
            input_is_string=False
        ),
        cl.PostcodeComparison(
            "owner_postal_code",
        ),
        cl.JaroWinklerAtThresholds(
            "owner_address",
        ),
        cl.NameComparison(
            "owner_firstname",
        ),
        cl.NameComparison(
            "owner_lastname",
        ),
    ],
)

linker = Linker(
    [df_prod, df_ff],
    settings,
    db_api=DuckDBAPI(),
    input_table_aliases=["df_prod", "df_ff"],
)

In [26]:
linker.training.estimate_probability_two_random_records_match(
    [block_on("matching_id")],
    recall=0.7,
)


Probability two random records match is estimated to be  6.25e-07.
This means that amongst all possible pairwise record comparisons, one in 1,599,263.40 are expected to match.  With 5,219,680,454,244 total possible comparisons, we expect a total of around 3,263,802.86 matching pairs


In [27]:
linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("matching_id")
)

SplinkException: Error executing the following sql for table `__splink__df_concat_with_tf`(__splink__df_concat_with_tf_a2481cf80):
CREATE TABLE __splink__df_concat_with_tf_a2481cf80 AS
WITH __splink__df_concat AS (
  SELECT
    'df_prod' AS source_dataset,
    "unique_id",
    "matching_id",
    "owner_fullname",
    "owner_birth_date",
    "owner_postal_code",
    "owner_address",
    "owner_firstname",
    "owner_lastname",
    RAND() AS __splink_salt
  FROM df_prod
  UNION ALL
  SELECT
    'df_ff' AS source_dataset,
    "unique_id",
    "matching_id",
    "owner_fullname",
    "owner_birth_date",
    "owner_postal_code",
    "owner_address",
    "owner_firstname",
    "owner_lastname",
    RAND() AS __splink_salt
  FROM df_ff
), __splink__df_tf_owner_fullname AS (
  SELECT
    "owner_fullname",
    CAST(COUNT(*) AS DOUBLE) / NULLIF((
      SELECT
        COUNT("owner_fullname") AS total
      FROM __splink__df_concat
    ), 0) AS "tf_owner_fullname"
  FROM __splink__df_concat
  WHER

In [32]:
from splink.exploratory import similarity_analysis as sa

In [ ]:
df

In [34]:
df_ff_owners

,id,ff_owner_idpersonne,local_ids,full_name,birth_date,postal_code,postal_code_1,ipo.address_dgfip[0],ff_owner_fullname,ff_owner_birth_date,ff_owner_firstname,ff_owner_lastname,ff_owner_postal_code,owner_address,matching_id
0,9275e4bc-681f-417d-89cb-2071a1268d10,13MDJGCJ,"[130420013315, 130420013315]",TONNA MARIE-ANGE,1963-09-12 01:00:00+01:00,,,None,TONNA/CATHERINE AGNES,1966-03-20,CATHERINE,BAIGNOL,83330,None,2b2c0ad0-171d-4b54-83ba-a93da85c7045
1,05fbd43b-c4bb-4f16-84fb-31139aed364d,13PBL2G4,"[130430379792, 130430379792]",MME BONNABEL MONIQUE,1953-01-15 01:00:00+01:00,,,None,CLEMAR,NaT,None,None,13410,None,8efb232c-fc9d-4f19-8861-94196b5496a8
2,bfb1999c-caf6-4b18-a5fe-e9062d80d9df,13PBLR69,"[130440035941, 130440035941]",DE CHAMPEAUX DE LA BOULAYE ERIC ALBERT JAUFFRET,1969-01-30 01:00:00+01:00,,,None,SCEA DOMAINE DE L'ENCLOS,NaT,None,None,13450,None,39b28dd7-02a1-4b79-bf0a-a556f09a535e
3,db6c0a00-1e32-4ad7-aca8-85da53ba27c1,13MCT6LX,"[131000247769, 131000247769]",CHOTEL THIBAULT CONSTANTIN REMY,1996-07-16 02:00:00+02:00,,,None,CHOTEL/THIBAULT CONSTANTIN REMY,1996-07-16,THIBAULT,CHOTEL,12800,None,dabbc494-9695-4ccc-8a2f-3d1e99f754f2
4,9dd6da3d-3030-454e-88b9-7a1fcec9a2fb,13MD2B3D,"[130530408463, 130530408463]",M LECAILLON BAPTISTIN CLOVIS AUGUSTIN,1994-03-29 02:00:00+02:00,,,None,LECAILLON/BAPTISTIN CLOVIS AUGUSTIN,1994-03-29,BAPTISTIN CLOVIS AUGUSTIN,LECAILLON,13370,None,cf204e68-9db1-4c1b-82dd-ce9bc25526f7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2284657,1c40484c-02f4-4c5c-a5f5-5195d40fb361,97MBC632,"[976110028925, 976110028925]",M HAMADA TARMIDHI,1977-10-06 01:00:00+01:00,,,None,HAMADA/TARMIDHI,1977-10-06,TARMIDHI,HAMADA,97600,None,7fd46ae2-e18d-4beb-8391-f71073ede725
2284658,6a7928a9-5fd8-4d73-a627-cd75c169e665,97MBDFWX,"[976160015705, 976160015705]",ABDOU SOURAYA,1985-05-10 02:00:00+02:00,,,None,ABDOU/SOURAYA,1985-05-10,SOURAYA,ABDOU,97640,None,02fe6513-2d56-4726-8b3e-6783702f86f5
2284659,d22a08e9-181b-4520-a2f1-abf83752b88e,97MBMJBK,"[974230326217, 974230326217]",POONOOSAWMY GEORGES-MARIE,1969-10-08 01:00:00+01:00,,,None,POONOOSAWMY/JEROME OLIVIER,1981-07-05,JEROME OLIVIER,POONOOSAWMY,97400,None,0eb396d2-0c9b-4e77-8a08-aad44f3b06fe
2284660,ee6d5701-7c89-43a8-b313-5fd9c838a2de,97MBDFWX,"[976160015705, 976160015705]",MME ABDOU MOINAMAOULIDA,1989-10-07 01:00:00+01:00,,,None,ABDOU/SOURAYA,1985-05-10,SOURAYA,ABDOU,97640,None,e6e29ca3-05a6-41da-9c3d-6bac96c3a26f


In [ ]:
from rapidfuzz import fuzz
from tqdm.auto import tqdm


def calculate_similarity(row):
    scores = {
        'full_name': fuzz.ratio(row['full_name'], row['ff_owner_fullname']),
        'full_name_first_name': 50 if row["ff_owner_firstname"] in row["full_name"] else 0,
        'full_name_last_name': 50 if row["ff_owner_lastname"] in row["full_name"] else 0,
        'birth_date': 100 if row['birth_date'] == row['ff_owner_birth_date'] else 0,
        'postal_code': fuzz.ratio(row['postal_code'], row['ff_owner_postal_code']),
        'address': fuzz.ratio(row['ipo.address_dgfip[0]'], row['owner_address']),
    }
    
    weights = {
        'full_name': 0.7,
        'birth_date': 0.9,
        'full_name_first_name': 0.2,
        'full_name_last_name': 0.2,
        'postal_code': 0.7,
        'address': 0.8,
    }
    
    global_score = sum(scores[k] * weights[k] for k in scores) / sum(weights.values())
    return global_score


# Ajouter une colonne avec le score de similarité
df_ff_owners['similarity_score'] = df_ff_owners.apply(calculate_similarity, axis=1)


In [16]:
df_ff_owners.columns

Index(['id', 'ff_owner_idpersonne', 'local_ids', 'full_name', 'birth_date',
       'postal_code', 'postal_code_1', 'ipo.address_dgfip[0]',
       'ff_owner_fullname', 'ff_owner_birth_date', 'ff_owner_firstname',
       'ff_owner_lastname', 'ff_owner_postal_code', 'owner_address'],
      dtype='object')